In [1]:
import pandas as pd
import numpy as np

Data Prep

In [2]:
# Data Loading

agency = pd.read_csv('Data/agency_master.xls')
users = pd.read_csv("Data/user_master.xls")
devices = pd.read_csv("Data/device_master.xls")
ips = pd.read_csv("Data/ip_master.xls")

booking = pd.read_csv("Data/booking_fact.xls")
labels = pd.read_csv("Data/booking_label_table.xls")
post_events = pd.read_csv("Data/post_booking_events.xls")

In [3]:
# Schema Inspection 

for name, df in {
    "agency": agency,
    "users": users,
    "devices": devices,
    "ips": ips,
    "booking": booking,
    "labels": labels,
    "post_events": post_events
}.items():
    print("\n", name)
    print(df.shape)
    print(df.columns)


 agency
(200, 6)
Index(['agency_id', 'country', 'agency_age_days', 'kyc_status', 'credit_limit',
       'status'],
      dtype='object')

 users
(800, 9)
Index(['user_id', 'agency_id', 'role', 'user_age_days', 'avg_logins_per_day',
       'failed_login_ratio', 'account_status', 'user_fraud_label',
       'user_fraud_type'],
      dtype='object')

 devices
(500, 5)
Index(['device_id', 'device_type', 'os', 'browser', 'first_seen_ts'], dtype='object')

 ips
(600, 6)
Index(['ip_id', 'ip_address', 'country', 'asn', 'is_proxy', 'first_seen_ts'], dtype='object')

 booking
(8000, 15)
Index(['booking_id', 'booking_ts', 'agency_id', 'user_id', 'device_id',
       'ip_id', 'product_type', 'route_type', 'origin_country', 'dest_country',
       'lead_time_days', 'booking_value', 'passengers_count', 'payment_method',
       'booking_status'],
      dtype='object')

 labels
(8000, 3)
Index(['booking_id', 'fraud_label', 'fraud_reason'], dtype='object')

 post_events
(8000, 8)
Index(['event_id', 'book

In [4]:
ips = ips.rename(columns={"first_seen_ts": "ips_first_seen_ts"})
devices = devices.rename(columns = {"first_seen_ts":"devices_first_seen_ts"})

agency = agency.rename(columns={"country":"agency_country"})
ips = ips.rename(columns={"country":"ips_country"})

In [5]:
# Merging Tables

df = booking.merge(agency, on="agency_id", how="left")
df = df.merge(users, on="user_id", how="left")
df = df.merge(devices, on="device_id", how="left")
df = df.merge(ips, on="ip_id", how="left")
df = df.merge(post_events, on="booking_id", how="left")

In [6]:
# Merge Health

df.isnull().mean().sort_values(ascending=False).head(20)

booking_id               0.0
booking_ts               0.0
failed_login_ratio       0.0
account_status           0.0
user_fraud_label         0.0
user_fraud_type          0.0
device_type              0.0
os                       0.0
browser                  0.0
devices_first_seen_ts    0.0
ip_address               0.0
ips_country              0.0
asn                      0.0
is_proxy                 0.0
ips_first_seen_ts        0.0
event_id                 0.0
is_cancelled             0.0
cancel_delay_days        0.0
is_disputed              0.0
dispute_delay_days       0.0
dtype: float64

In [7]:
[c for c in df.columns if c.endswith("_x") or c.endswith("_y")]

['agency_id_x', 'agency_id_y']

In [8]:
# Sanity Checks

#df["booking_id"].nunique() == len(df)
#df["device_id"].value_counts().head()
#df["agency_id"].value_counts().head()

In [9]:
for col in ["agency_id"]:
    df[col] = df[f"{col}_y"]  # or _x depending on preference

df = df.drop(columns=[
    "agency_id_x","agency_id_y", 'user_fraud_label',
       'user_fraud_type'
])

In [10]:
# df.to_csv("merged.csv")

In [11]:
df.shape

(8000, 41)

In [12]:
# Pickle file
df.to_pickle("data/processed/merged.pkl")